<h1>Battle of the Neighborhoods</h1>
<h2>Introduction: Business Problem</h2>
<p>In this project I will try to find an optimal location for a taco restaurant. There are lots of restaurants in Toronto we will try to detect locations that are not already crowded with restaurants. I will also try to find count of possible customers while finding count of schools and offices, after it we will create success rate which show count of people per taco restaurants.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.</p>

<h2>Approach</h2>
<p>Based on definition of our problem, we have to complate these steps:</p>
<ul>
    <li>Finding cordinates of possible locations in Toronto</li>
    <li>Finding count of taco restaurants for the locations</li>
    <li>Finding count of offices for the locations</li>
    <li>Finding count of schools for the locations</li>
    <li>Merging th datas and cleaning it from null and dublicates values</li>
    <li>Find the suitable places with help of k-means Clustering</li>
 </ul>

<h2>Finding cordinates of possible locations in Toronto</h2>
<h2>----------------------------------------------------------------------------------------------------------------------------</h2>

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
data  = requests.get(url).text 

In [4]:
soup = BeautifulSoup(data,"html5lib")
table = soup.find('table')

data= ""
dfd = []
thedict = {}

for row in table.find_all('tr'): 
    cols = row.find_all('td')
    postal_code = '';
    if len(cols) > 2 :
        postal_code = str(cols[0].getText().rstrip())
        borough = cols[1].getText().rstrip() 
        neighbourhood= cols[2].getText().rstrip()
        if postal_code != 'Not assigned' and borough != 'Not assigned' and neighbourhood != 'Not assigned':
            
            
           
                
            try:
                print(thedict[postal_code])
            except:
                arrt=[postal_code,borough,neighbourhood]
                thedict[postal_code] = arrt
             
for x in thedict.keys():
    dfd.append(thedict[x])

             
        
df = pd.DataFrame(dfd, columns = ['Postal Code','Borough','Neighbourhood'])

In [5]:
print(df.shape)

(103, 3)


In [6]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
coors = pd.read_csv('toronto_coordinates.csv')

Coordinates downloaded!


In [7]:
df = pd.merge(df, coors, how='left', left_on = 'Postal Code', right_on = 'Postal Code')
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [8]:
from geopy.geocoders import Nominatim

# get the corrdinates of Toronto, Canada
address = "Toronto, ON"
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('lat =',latitude,', long = ',longitude,').')

lat = 43.6534817 , long =  -79.3839347 ).


In [9]:
!pip install folium

     |████████████████████████████████| 93 kB 4.0 MB/s  eta 0:00:01


In [10]:
import folium

In [11]:
simple_map = folium.Map(location=[latitude, longitude], zoom_start=10)
simple_map

In [12]:
for lat, long, borough, neighbourhood in zip(
        df['Latitude'], 
        df['Longitude'], 
        df['Borough'], 
        df['Neighbourhood']):
    # format label as "neighbourhood, borough" and make it a pop up
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    # make a green circle for each point
    folium.CircleMarker(
        [lat, long],
        radius=4,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='blue',
        fill_opacity=0.8,
        parse_html=False).add_to(simple_map) 

In [13]:
simple_map


<h2>----------------------------------------------------------------------------------------------------------------------------</h2>

<h2>Finding count of taco restaurants for the locations</h2>
<h2>----------------------------------------------------------------------------------------------------------------------------</h2>

In [14]:
from pandas import json_normalize

CLIENT_ID = 'secret'
CLIENT_SECRET = 'secret'
VERSION = '20180605'

In [16]:
    def getPlaces(names, latitudes, longitudes, search_query,radius=100,  df=df, pd = pd,):
        venues_list=[]
        limit=100
        radius=500

        for name, lat, lng in zip(names, latitudes, longitudes):

            url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
                CLIENT_ID,
                CLIENT_SECRET,
                lat,
                lng,
                VERSION,
                search_query,
                radius,
                limit)
            
            results = requests.get(url).json()["response"]['venues']
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['name'], 
                v['location']['lat'], 
                v['location']['lng'],  
                v['categories']) for v in results])

        places = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        places.columns = ['Neighborhood', 
                      'Neighborhood Latitude', 
                      'Neighborhood Longitude', 
                      'Venue', 
                      'Venue Latitude', 
                      'Venue Longitude', 
                      'Venue Category']

        return(places)

In [17]:
taco_shops = getPlaces(names=df['Neighbourhood'],latitudes=df['Latitude'],longitudes=df['Longitude'], search_query='taco', radius=100)
taco_shops.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Taco Bell,43.661032,-79.383713,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R..."
1,"Garden District, Ryerson",43.657162,-79.378937,Taco Bell / KFC,43.654883,-79.380440,[]
2,"Garden District, Ryerson",43.657162,-79.378937,Taco Bell,43.661032,-79.383713,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R..."
3,"Garden District, Ryerson",43.657162,-79.378937,Taco Villa,43.652677,-79.380227,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M..."
4,"Garden District, Ryerson",43.657162,-79.378937,Richtree Tacos & Burritos,43.652602,-79.380201,"[{'id': '4bf58dd8d48988d151941735', 'name': 'T..."


In [18]:
taco_shops

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Taco Bell,43.661032,-79.383713,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R..."
1,"Garden District, Ryerson",43.657162,-79.378937,Taco Bell / KFC,43.654883,-79.380440,[]
2,"Garden District, Ryerson",43.657162,-79.378937,Taco Bell,43.661032,-79.383713,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R..."
3,"Garden District, Ryerson",43.657162,-79.378937,Taco Villa,43.652677,-79.380227,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M..."
4,"Garden District, Ryerson",43.657162,-79.378937,Richtree Tacos & Burritos,43.652602,-79.380201,"[{'id': '4bf58dd8d48988d151941735', 'name': 'T..."
5,St. James Town,43.651494,-79.375418,Taco Villa,43.652677,-79.380227,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M..."
6,St. James Town,43.651494,-79.375418,Quesada Burritos & Tacos,43.650016,-79.374627,"[{'id': '56aa371ae4b08b9a8d5734ba', 'name': 'T..."
7,St. James Town,43.651494,-79.375418,KFC,43.649300,-79.379400,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F..."
8,St. James Town,43.651494,-79.375418,Richtree Tacos & Burritos,43.652602,-79.380201,"[{'id': '4bf58dd8d48988d151941735', 'name': 'T..."
9,Central Bay Street,43.657952,-79.387383,Taco Bell,43.661032,-79.383713,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R..."


In [19]:
def get_category_type(row):
    try:
        categories_list = row['Venue Category']
    except:
        categories_list = row['venue.Venue Category']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
filt = taco_shops
filt['Venue Category'] = filt.apply(get_category_type, axis=1)

filtc = filt[filt['Venue Category'] != 'Fast Food Restaurant']
filtc = filtc[filtc['Venue Category'] != 'Fried Chicken Joint']
filtc.head(100)

tcn = filtc
del tcn['Neighborhood Latitude']
del tcn['Neighborhood Longitude']
del tcn['Venue Latitude']
del tcn['Venue Longitude']
del tcn['Venue Category']
taco_count = tcn.rename(columns={'Venue': 'Restaurant Count'})
final_df = taco_count.groupby('Neighborhood').count()
final_df


,Restaurant Count
Neighborhood,
Central Bay Street,4
Church and Wellesley,3
"Commerce Court, Victoria Hotel",3
"Dufferin, Dovercourt Village",1
"East Toronto, Broadview North (Old East York)",1
"Fairview, Henry Farm, Oriole",1
"First Canadian Place, Underground city",3
"Garden District, Ryerson",4
"Harbourfront East, Union Station, Toronto Islands",2


In [21]:
final_df

,Restaurant Count
Neighborhood,
Central Bay Street,4
Church and Wellesley,3
"Commerce Court, Victoria Hotel",3
"Dufferin, Dovercourt Village",1
"East Toronto, Broadview North (Old East York)",1
"Fairview, Henry Farm, Oriole",1
"First Canadian Place, Underground city",3
"Garden District, Ryerson",4
"Harbourfront East, Union Station, Toronto Islands",2


<h2>----------------------------------------------------------------------------------------------------------------------------</h2>

<h2>Finding count of offices for the locations</h2>
<h2>----------------------------------------------------------------------------------------------------------------------------</h2>


In [22]:
offices = getPlaces(names=df['Neighbourhood'],latitudes=df['Latitude'],longitudes=df['Longitude'], search_query='office', radius=100)
offices.head(100)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Victoria Village,43.725882,-79.315572,Oasis Dental Office,43.725950,-79.312322,[]
1,Victoria Village,43.725882,-79.315572,Beck Taxi Head Offices,43.725883,-79.318173,"[{'id': '54541b70498ea6ccd0204bff', 'name': 'T..."
2,"Regent Park, Harbourfront",43.654260,-79.360636,iRunRealEstate Corktown-Distillery Office,43.652783,-79.363775,"[{'id': '5032885091d4c4b30a586d66', 'name': 'R..."
3,"Regent Park, Harbourfront",43.654260,-79.360636,Brian Chang Campaign Office (ChangQ),43.655601,-79.364993,"[{'id': '5032764e91d4c4b30a586d5a', 'name': 'C..."
4,"Regent Park, Harbourfront",43.654260,-79.360636,Toronto Centre Riding Office,43.653162,-79.363482,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G..."
...,...,...,...,...,...,...,...
95,"Garden District, Ryerson",43.657162,-79.378937,The Doctor's Office,43.656220,-79.383828,"[{'id': '4bf58dd8d48988d177941735', 'name': 'D..."
96,"Garden District, Ryerson",43.657162,-79.378937,Air Canada Sales Office,43.653362,-79.378269,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O..."
97,"Garden District, Ryerson",43.657162,-79.378937,Flight Centre Toronto Head Office,43.655957,-79.382049,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O..."
98,"Garden District, Ryerson",43.657162,-79.378937,Dr. Michael Sved's Office - OBGYN,43.658123,-79.384702,"[{'id': '4bf58dd8d48988d104941735', 'name': 'M..."


In [23]:
offices_ = offices
offices_['Venue Category'] = offices.apply(get_category_type, axis=1)

offices_

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Victoria Village,43.725882,-79.315572,Oasis Dental Office,43.725950,-79.312322,None
1,Victoria Village,43.725882,-79.315572,Beck Taxi Head Offices,43.725883,-79.318173,Transportation Service
2,"Regent Park, Harbourfront",43.654260,-79.360636,iRunRealEstate Corktown-Distillery Office,43.652783,-79.363775,Real Estate Office
3,"Regent Park, Harbourfront",43.654260,-79.360636,Brian Chang Campaign Office (ChangQ),43.655601,-79.364993,Campaign Office
4,"Regent Park, Harbourfront",43.654260,-79.360636,Toronto Centre Riding Office,43.653162,-79.363482,Government Building
...,...,...,...,...,...,...,...
830,Church and Wellesley,43.665860,-79.383160,"Hoyes, Michalos & Associates Inc. – Consumer P...",43.670914,-79.386907,Financial or Legal Service
831,Church and Wellesley,43.665860,-79.383160,Rosetta,43.669956,-79.386281,Office
832,Church and Wellesley,43.665860,-79.383160,Butler Ruston Bell Offices,43.668046,-79.386273,Office
833,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Absolutely Love It - Head Office,43.664697,-79.321838,Coworking Space


In [24]:
ton = offices_

del ton['Neighborhood Latitude']
del ton['Neighborhood Longitude']
del ton['Venue Latitude']
del ton['Venue Longitude']
del ton['Venue Category']

office_count = ton.rename(columns={'Venue': 'Office Count'})
office_countf = office_count.groupby('Neighborhood').count()
office_countf

,Office Count
Neighborhood,
"Alderwood, Long Branch",4
"Bathurst Manor, Wilson Heights, Downsview North",3
"Bedford Park, Lawrence Manor East",4
Berczy Park,50
"Birch Cliff, Cliffside West",1
...,...
Weston,1
"Wexford, Maryvale",1
"Willowdale, Willowdale East",13


In [26]:
office_countf.head(71)

,Office Count
Neighborhood,
"Alderwood, Long Branch",4
"Bathurst Manor, Wilson Heights, Downsview North",3
"Bedford Park, Lawrence Manor East",4
Berczy Park,50
"Birch Cliff, Cliffside West",1
...,...
Weston,1
"Wexford, Maryvale",1
"Willowdale, Willowdale East",13


In [27]:
final_dfback = final_df
final_df = pd.concat([final_df,office_countf], axis=1, sort=False)
final_df

,Restaurant Count,Office Count
Central Bay Street,4.0,50.0
Church and Wellesley,3.0,47.0
"Commerce Court, Victoria Hotel",3.0,50.0
"Dufferin, Dovercourt Village",1.0,4.0
"East Toronto, Broadview North (Old East York)",1.0,NaN
...,...,...
Weston,NaN,1.0
"Wexford, Maryvale",NaN,1.0
"Willowdale, Willowdale East",NaN,13.0
Woodbine Heights,NaN,2.0


<h2>----------------------------------------------------------------------------------------------------------------------------</h2>

<h2>Finding count of schools for the locations</h2>
<h2>----------------------------------------------------------------------------------------------------------------------------</h2>


In [28]:
schools = getPlaces(names=df['Neighbourhood'],latitudes=df['Latitude'],longitudes=df['Longitude'], search_query='school', radius=100)
schools.head(100)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Cassandra Public School Pring Fling,43.748443,-79.328239,"[{'id': '4f4533804b9074f6e4fb0105', 'name': 'E..."
1,Victoria Village,43.725882,-79.315572,North Toronto Christian School,43.722998,-79.314543,"[{'id': '4bf58dd8d48988d13b941735', 'name': 'S..."
2,Victoria Village,43.725882,-79.315572,Some School In Scarbourogh,43.722861,-79.312567,[]
3,"Regent Park, Harbourfront",43.654260,-79.360636,George Brown College - School of ESL,43.651872,-79.365580,"[{'id': '4bf58dd8d48988d198941735', 'name': 'C..."
4,"Regent Park, Harbourfront",43.654260,-79.360636,Inglenook Community School,43.653903,-79.359429,"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H..."
...,...,...,...,...,...,...,...
95,The Beaches,43.676357,-79.293031,Balmy Beach School,43.676199,-79.290134,"[{'id': '4bf58dd8d48988d13b941735', 'name': 'S..."
96,The Beaches,43.676357,-79.293031,St. Denis Catholic School,43.672881,-79.290056,"[{'id': '4bf58dd8d48988d13b941735', 'name': 'S..."
97,The Beaches,43.676357,-79.293031,St. John's School Playground,43.680739,-79.296430,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P..."
98,The Beaches,43.676357,-79.293031,Glen Ames Middle School,43.674155,-79.298477,"[{'id': '4f4533814b9074f6e4fb0106', 'name': 'M..."


In [29]:
schools_ = schools
schools_['Venue Category'] = schools.apply(get_category_type, axis=1)

print(schools_)

                                          Neighborhood  Neighborhood Latitude  \
0                                            Parkwoods              43.753259   
1                                     Victoria Village              43.725882   
2                                     Victoria Village              43.725882   
3                            Regent Park, Harbourfront              43.654260   
4                            Regent Park, Harbourfront              43.654260   
..                                                 ...                    ...   
383                               Church and Wellesley              43.665860   
384  Old Mill South, King's Mill Park, Sunnylea, Hu...              43.636258   
385  Old Mill South, King's Mill Park, Sunnylea, Hu...              43.636258   
386  Mimico NW, The Queensway West, South of Bloor,...              43.628841   
387  Mimico NW, The Queensway West, South of Bloor,...              43.628841   

     Neighborhood Longitude

In [30]:
ton = schools_

del ton['Neighborhood Latitude']
del ton['Neighborhood Longitude']
del ton['Venue Latitude']
del ton['Venue Longitude']
del ton['Venue Category']
schools_count = ton.rename(columns={'Venue': 'School Count'})
schools_count_f = schools_count.groupby('Neighborhood').count()

print(schools_count_f.head(100))

                                                 School Count
Neighborhood                                                 
Agincourt                                                   1
Alderwood, Long Branch                                      2
Bathurst Manor, Wilson Heights, Downsview North             1
Bayview Village                                             2
Bedford Park, Lawrence Manor East                           2
...                                                       ...
Willowdale, Willowdale West                                 2
Woburn                                                      1
Woodbine Heights                                            3
York Mills West                                             2
York Mills, Silver Hills                                    4

[86 rows x 1 columns]


In [32]:

schools_count_f.head(100)

,School Count
Neighborhood,
Agincourt,1
"Alderwood, Long Branch",2
"Bathurst Manor, Wilson Heights, Downsview North",1
Bayview Village,2
"Bedford Park, Lawrence Manor East",2
...,...
"Willowdale, Willowdale West",2
Woburn,1
Woodbine Heights,3


<h2>----------------------------------------------------------------------------------------------------------------------------</h2>

<h2>Merging th datas and cleaning it from null and dublicates values</h2>
<h2>----------------------------------------------------------------------------------------------------------------------------</h2>


In [33]:
final_df = pd.concat([final_df,schools_count_f], axis=1, sort=False)
final_df

,Restaurant Count,Office Count,School Count
Central Bay Street,4.0,50.0,23.0
Church and Wellesley,3.0,47.0,10.0
"Commerce Court, Victoria Hotel",3.0,50.0,9.0
"Dufferin, Dovercourt Village",1.0,4.0,3.0
"East Toronto, Broadview North (Old East York)",1.0,NaN,2.0
...,...,...,...
Scarborough Village,NaN,NaN,1.0
"Willowdale, Newtonbrook",NaN,NaN,1.0
"Willowdale, Willowdale West",NaN,NaN,2.0
Woburn,NaN,NaN,1.0


In [34]:

final_df['Restaurant Count'] = final_df['Restaurant Count'].fillna(0)

In [35]:

final_df['Office Count'] = final_df['Office Count'].fillna(0)

In [36]:
final_df['School Count'] = final_df['School Count'].fillna(0)

In [37]:
#we tahe that approximatelly people at shcool 20 times much more than people at offices.
final_df['Population'] = final_df['School Count']*20 + final_df['Office Count'];
final_df

,Restaurant Count,Office Count,School Count,Population
Central Bay Street,4.0,50.0,23.0,510.0
Church and Wellesley,3.0,47.0,10.0,247.0
"Commerce Court, Victoria Hotel",3.0,50.0,9.0,230.0
"Dufferin, Dovercourt Village",1.0,4.0,3.0,64.0
"East Toronto, Broadview North (Old East York)",1.0,0.0,2.0,40.0
...,...,...,...,...
Scarborough Village,0.0,0.0,1.0,20.0
"Willowdale, Newtonbrook",0.0,0.0,1.0,20.0
"Willowdale, Willowdale West",0.0,0.0,2.0,40.0
Woburn,0.0,0.0,1.0,20.0


<h4> I create another field "success rate", and calculate count of places per taco restaurant</h4>

In [38]:
final_df['success rate'] = final_df['Population']/(final_df['Restaurant Count']+1)
final_df

,Restaurant Count,Office Count,School Count,Population,success rate
Central Bay Street,4.0,50.0,23.0,510.0,102.00
Church and Wellesley,3.0,47.0,10.0,247.0,61.75
"Commerce Court, Victoria Hotel",3.0,50.0,9.0,230.0,57.50
"Dufferin, Dovercourt Village",1.0,4.0,3.0,64.0,32.00
"East Toronto, Broadview North (Old East York)",1.0,0.0,2.0,40.0,20.00
...,...,...,...,...,...
Scarborough Village,0.0,0.0,1.0,20.0,20.00
"Willowdale, Newtonbrook",0.0,0.0,1.0,20.0,20.00
"Willowdale, Willowdale West",0.0,0.0,2.0,40.0,40.00
Woburn,0.0,0.0,1.0,20.0,20.00


In [42]:
final_df = final_df.sort_values(by=['success rate'], ascending=False)
final_df.head(100)

,Restaurant Count,Office Count,School Count,Population,success rate
"University of Toronto, Harbord",0.0,23.0,13.0,283.0,283.0
"Regent Park, Harbourfront",0.0,10.0,12.0,250.0,250.0
"Willowdale, Willowdale East",0.0,13.0,10.0,213.0,213.0
Davisville,0.0,15.0,9.0,195.0,195.0
Berczy Park,0.0,50.0,7.0,190.0,190.0
...,...,...,...,...,...
"Dorset Park, Wexford Heights, Scarborough Town Centre",0.0,1.0,0.0,1.0,1.0
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",0.0,1.0,0.0,1.0,1.0
Glencairn,0.0,1.0,0.0,1.0,1.0
"Rouge Hill, Port Union, Highland Creek",0.0,1.0,0.0,1.0,1.0


In [43]:
final_df_back = final_df

In [62]:
final_df = final_df_back
final_df

,Restaurant Count,Office Count,School Count,Population,success rate,Neighborhood,Neighbourhood
0,0.0,23.0,13.0,283.0,283.0,"University of Toronto, Harbord",0
1,0.0,10.0,12.0,250.0,250.0,"Regent Park, Harbourfront",1
2,0.0,13.0,10.0,213.0,213.0,"Willowdale, Willowdale East",2
3,0.0,15.0,9.0,195.0,195.0,Davisville,3
4,0.0,50.0,7.0,190.0,190.0,Berczy Park,4
...,...,...,...,...,...,...,...
89,0.0,1.0,0.0,1.0,1.0,"Dorset Park, Wexford Heights, Scarborough Town...",89
90,0.0,1.0,0.0,1.0,1.0,"Business reply mail Processing Centre, South C...",90
91,0.0,1.0,0.0,1.0,1.0,Glencairn,91
92,0.0,1.0,0.0,1.0,1.0,"Rouge Hill, Port Union, Highland Creek",92


In [65]:

final_df['Neighbourhood'] = final_df['Neighborhood']
final_df = final_df.drop('Neighborhood',1)
final_df




,Restaurant Count,Office Count,School Count,Population,success rate,Neighbourhood
0,0.0,23.0,13.0,283.0,283.0,"University of Toronto, Harbord"
1,0.0,10.0,12.0,250.0,250.0,"Regent Park, Harbourfront"
2,0.0,13.0,10.0,213.0,213.0,"Willowdale, Willowdale East"
3,0.0,15.0,9.0,195.0,195.0,Davisville
4,0.0,50.0,7.0,190.0,190.0,Berczy Park
...,...,...,...,...,...,...
89,0.0,1.0,0.0,1.0,1.0,"Dorset Park, Wexford Heights, Scarborough Town..."
90,0.0,1.0,0.0,1.0,1.0,"Business reply mail Processing Centre, South C..."
91,0.0,1.0,0.0,1.0,1.0,Glencairn
92,0.0,1.0,0.0,1.0,1.0,"Rouge Hill, Port Union, Highland Creek"


In [66]:
final_df.reset_index(inplace=True,drop=True)
final_df



,Restaurant Count,Office Count,School Count,Population,success rate,Neighbourhood
0,0.0,23.0,13.0,283.0,283.0,"University of Toronto, Harbord"
1,0.0,10.0,12.0,250.0,250.0,"Regent Park, Harbourfront"
2,0.0,13.0,10.0,213.0,213.0,"Willowdale, Willowdale East"
3,0.0,15.0,9.0,195.0,195.0,Davisville
4,0.0,50.0,7.0,190.0,190.0,Berczy Park
...,...,...,...,...,...,...
89,0.0,1.0,0.0,1.0,1.0,"Dorset Park, Wexford Heights, Scarborough Town..."
90,0.0,1.0,0.0,1.0,1.0,"Business reply mail Processing Centre, South C..."
91,0.0,1.0,0.0,1.0,1.0,Glencairn
92,0.0,1.0,0.0,1.0,1.0,"Rouge Hill, Port Union, Highland Creek"


In [74]:
df_report= pd.merge(final_df,df, on=['Neighbourhood'])
df_report

,Restaurant Count,Office Count,School Count,Population,success rate,Neighbourhood,Postal Code,Borough,Latitude,Longitude
0,0.0,23.0,13.0,283.0,283.0,"University of Toronto, Harbord",M5S,Downtown Toronto,43.662696,-79.400049
1,0.0,10.0,12.0,250.0,250.0,"Regent Park, Harbourfront",M5A,Downtown Toronto,43.654260,-79.360636
2,0.0,13.0,10.0,213.0,213.0,"Willowdale, Willowdale East",M2N,North York,43.770120,-79.408493
3,0.0,15.0,9.0,195.0,195.0,Davisville,M4S,Central Toronto,43.704324,-79.388790
4,0.0,50.0,7.0,190.0,190.0,Berczy Park,M5E,Downtown Toronto,43.644771,-79.373306
...,...,...,...,...,...,...,...,...,...,...
93,0.0,1.0,0.0,1.0,1.0,"Dorset Park, Wexford Heights, Scarborough Town...",M1P,Scarborough,43.757410,-79.273304
94,0.0,1.0,0.0,1.0,1.0,"Business reply mail Processing Centre, South C...",M7Y,East Toronto,43.662744,-79.321558
95,0.0,1.0,0.0,1.0,1.0,Glencairn,M6B,North York,43.709577,-79.445073
96,0.0,1.0,0.0,1.0,1.0,"Rouge Hill, Port Union, Highland Creek",M1C,Scarborough,43.784535,-79.160497


In [75]:
df_report

,Restaurant Count,Office Count,School Count,Population,success rate,Neighbourhood,Postal Code,Borough,Latitude,Longitude
0,0.0,23.0,13.0,283.0,283.0,"University of Toronto, Harbord",M5S,Downtown Toronto,43.662696,-79.400049
1,0.0,10.0,12.0,250.0,250.0,"Regent Park, Harbourfront",M5A,Downtown Toronto,43.654260,-79.360636
2,0.0,13.0,10.0,213.0,213.0,"Willowdale, Willowdale East",M2N,North York,43.770120,-79.408493
3,0.0,15.0,9.0,195.0,195.0,Davisville,M4S,Central Toronto,43.704324,-79.388790
4,0.0,50.0,7.0,190.0,190.0,Berczy Park,M5E,Downtown Toronto,43.644771,-79.373306
...,...,...,...,...,...,...,...,...,...,...
93,0.0,1.0,0.0,1.0,1.0,"Dorset Park, Wexford Heights, Scarborough Town...",M1P,Scarborough,43.757410,-79.273304
94,0.0,1.0,0.0,1.0,1.0,"Business reply mail Processing Centre, South C...",M7Y,East Toronto,43.662744,-79.321558
95,0.0,1.0,0.0,1.0,1.0,Glencairn,M6B,North York,43.709577,-79.445073
96,0.0,1.0,0.0,1.0,1.0,"Rouge Hill, Port Union, Highland Creek",M1C,Scarborough,43.784535,-79.160497


<h2>----------------------------------------------------------------------------------------------------------------------------</h2>

<h2>Find the suitable places with help of k-means Clustering</h2>
<h2>----------------------------------------------------------------------------------------------------------------------------</h2>


In [72]:
from sklearn.cluster import KMeans

In [76]:
df_report_backup = df_report
kclusters = 5
df_report = df_report.drop('Neighbourhood',1)
df_report = df_report.drop('Postal Code',1)
df_report = df_report.drop('Borough',1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_report)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 0, 0, 0, 0, 0], dtype=int32)

In [77]:
clustered = df_report_backup
clustered.insert(0,'Cluster Labels',kmeans.labels_)
clustered.head()

,Cluster Labels,Restaurant Count,Office Count,School Count,Population,success rate,Neighbourhood,Postal Code,Borough,Latitude,Longitude
0,2,0.0,23.0,13.0,283.0,283.0,"University of Toronto, Harbord",M5S,Downtown Toronto,43.662696,-79.400049
1,2,0.0,10.0,12.0,250.0,250.0,"Regent Park, Harbourfront",M5A,Downtown Toronto,43.654260,-79.360636
2,2,0.0,13.0,10.0,213.0,213.0,"Willowdale, Willowdale East",M2N,North York,43.770120,-79.408493
3,2,0.0,15.0,9.0,195.0,195.0,Davisville,M4S,Central Toronto,43.704324,-79.388790
4,2,0.0,50.0,7.0,190.0,190.0,Berczy Park,M5E,Downtown Toronto,43.644771,-79.373306


In [78]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [80]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        clustered['Latitude'], 
        clustered['Longitude'], 
        clustered['Neighbourhood'], 
        clustered['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
       
map_clusters

In [81]:
clustered.loc[clustered['Cluster Labels'] == 0]

,Cluster Labels,Restaurant Count,Office Count,School Count,Population,success rate,Neighbourhood,Postal Code,Borough,Latitude,Longitude
5,0,0.0,5.0,8.0,165.0,165.0,"Little Portugal, Trinity",M6J,West Toronto,43.647927,-79.419750
6,0,0.0,13.0,7.0,153.0,153.0,Don Mills,M3B,North York,43.745906,-79.352188
7,0,0.0,13.0,7.0,153.0,153.0,Don Mills,M3C,North York,43.725900,-79.340923
8,0,0.0,8.0,7.0,148.0,148.0,"The Danforth West, Riverdale",M4K,East Toronto,43.679557,-79.352188
9,0,0.0,3.0,7.0,143.0,143.0,"The Annex, North Midtown, Yorkville",M5R,Central Toronto,43.672710,-79.405678
10,0,0.0,1.0,7.0,141.0,141.0,The Beaches,M4E,East Toronto,43.676357,-79.293031
11,0,0.0,0.0,7.0,140.0,140.0,Downsview,M3K,North York,43.737473,-79.464763
12,0,0.0,0.0,7.0,140.0,140.0,Downsview,M3L,North York,43.739015,-79.506944
13,0,0.0,0.0,7.0,140.0,140.0,Downsview,M3M,North York,43.728496,-79.495697
14,0,0.0,0.0,7.0,140.0,140.0,Downsview,M3N,North York,43.761631,-79.520999


In [82]:
clustered.loc[clustered['Cluster Labels'] == 1]

,Cluster Labels,Restaurant Count,Office Count,School Count,Population,success rate,Neighbourhood,Postal Code,Borough,Latitude,Longitude
29,1,0.0,0.0,4.0,80.0,80.0,Lawrence Park,M4N,Central Toronto,43.728020,-79.388790
30,1,0.0,0.0,4.0,80.0,80.0,"York Mills, Silver Hills",M2L,North York,43.757490,-79.374714
31,1,0.0,11.0,3.0,71.0,71.0,"Brockton, Parkdale Village, Exhibition Place",M6K,West Toronto,43.636847,-79.428191
32,1,0.0,5.0,3.0,65.0,65.0,Christie,M6G,Downtown Toronto,43.669542,-79.422564
33,1,0.0,4.0,3.0,64.0,64.0,"Runnymede, Swansea",M6S,West Toronto,43.651571,-79.484450
...,...,...,...,...,...,...,...,...,...,...,...
93,1,0.0,1.0,0.0,1.0,1.0,"Dorset Park, Wexford Heights, Scarborough Town...",M1P,Scarborough,43.757410,-79.273304
94,1,0.0,1.0,0.0,1.0,1.0,"Business reply mail Processing Centre, South C...",M7Y,East Toronto,43.662744,-79.321558
95,1,0.0,1.0,0.0,1.0,1.0,Glencairn,M6B,North York,43.709577,-79.445073
96,1,0.0,1.0,0.0,1.0,1.0,"Rouge Hill, Port Union, Highland Creek",M1C,Scarborough,43.784535,-79.160497


In [83]:
clustered.loc[clustered['Cluster Labels'] == 2]

,Cluster Labels,Restaurant Count,Office Count,School Count,Population,success rate,Neighbourhood,Postal Code,Borough,Latitude,Longitude
0,2,0.0,23.0,13.0,283.0,283.0,"University of Toronto, Harbord",M5S,Downtown Toronto,43.662696,-79.400049
1,2,0.0,10.0,12.0,250.0,250.0,"Regent Park, Harbourfront",M5A,Downtown Toronto,43.654260,-79.360636
2,2,0.0,13.0,10.0,213.0,213.0,"Willowdale, Willowdale East",M2N,North York,43.770120,-79.408493
3,2,0.0,15.0,9.0,195.0,195.0,Davisville,M4S,Central Toronto,43.704324,-79.388790
4,2,0.0,50.0,7.0,190.0,190.0,Berczy Park,M5E,Downtown Toronto,43.644771,-79.373306


In [84]:
clustered.loc[clustered['Cluster Labels'] == 3]

,Cluster Labels,Restaurant Count,Office Count,School Count,Population,success rate,Neighbourhood,Postal Code,Borough,Latitude,Longitude
16,3,1.0,50.0,10.0,250.0,125.000000,Stn A PO Boxes,M5W,Downtown Toronto,43.646435,-79.374846
19,3,1.0,50.0,9.0,230.0,115.000000,"Toronto Dominion Centre, Design Exchange",M5K,Downtown Toronto,43.647177,-79.381576
20,3,1.0,28.0,10.0,228.0,114.000000,"Queen's Park, Ontario Provincial Government",M7A,Downtown Toronto,43.662301,-79.389494
28,3,3.0,50.0,14.0,330.0,82.500000,St. James Town,M5C,Downtown Toronto,43.651494,-79.375418
36,3,3.0,47.0,10.0,247.0,61.750000,Church and Wellesley,M4Y,Downtown Toronto,43.665860,-79.383160
42,3,3.0,50.0,9.0,230.0,57.500000,"Commerce Court, Victoria Hotel",M5L,Downtown Toronto,43.648198,-79.379817
43,3,3.0,50.0,9.0,230.0,57.500000,"First Canadian Place, Underground city",M5X,Downtown Toronto,43.648429,-79.382280
44,3,4.0,50.0,11.0,270.0,54.000000,"Richmond, Adelaide, King",M5H,Downtown Toronto,43.650571,-79.384568
68,3,5.0,21.0,10.0,221.0,36.833333,"Kensington Market, Chinatown, Grange Park",M5T,Downtown Toronto,43.653206,-79.400049


In [85]:
clustered.loc[clustered['Cluster Labels'] == 4]

,Cluster Labels,Restaurant Count,Office Count,School Count,Population,success rate,Neighbourhood,Postal Code,Borough,Latitude,Longitude
18,4,4.0,50.0,27.0,590.0,118.0,"Garden District, Ryerson",M5B,Downtown Toronto,43.657162,-79.378937
23,4,4.0,50.0,23.0,510.0,102.0,Central Bay Street,M5G,Downtown Toronto,43.657952,-79.387383


<h2>----------------------------------------------------------------------------------------------------------------------------</h2>

<h1>Conclusions</h1>
<h2>----------------------------------------------------------------------------------------------------------------------------</h2>

In [ ]:
<h2>Result</h2>
<p>Of the five clusters generated in this project, four of them have a ratio of taco restaurants to office 
and school buildings of roughly 1:1 or 2:1. Only one cluster (Cluster 4) has a drastically different ratio of 1:7 (or only 1 coffee shop per 7 office bulidings). 
This means that of the five clusters, cluster 2 has the most clear need for new coffee shops.
Therefore, the neighborhoods in this cluster would be a good place for an entrepreneur to open one up.</p>